# Implementação do esquema KEM-RSA-OAEP

## Descrição do Exercício

A ideia do exercício passa por criar toda uma classe em Python que seja capaz de implementar o esquema **KEM-RSA-OAEP**.

Dado que os algoritmos e a forma como funcionam são públicos, o grupo apenas precisou de compreender como cada um deles funciona, transformando essa ideia para modo **SageMath**.

## Descrição da Implementação

Estando feita estra triagem de informação inicial, o grupo penso desde logo na ideia de criar duas classes. Uma que respondesse aos pedidos do OAEP e outra que fizesse depois a continuidade em modo RSA, possibilitando dessa forma a criação das chaves pública e privada e toda a parte de crifragem e decifragem.

**Com a pesquisa necessária e com a ideia do funcionamento do algortimo em mente, estabelecem-se as seguintes implementacões:**

- **Passos da algoritmia do OAEP (Codificação e Descodificação):**

  - Para codificar (*padding*):

    1. Mensagens ficam com um padding de k1-zeros para ficarem com o tamanho $n - k0$ bits.
    2. *r* é uma string random de tamanho $k0$ bits.
    3. *G* expande os $k0$ bits do *r* para $n − k0$ bits.
    4. *X* = *m00...0* ⊕ $G(r)$
    5. *H* reduz os $n − k0$ bits do X para $k0$ bits.
    6. *Y* = *r* ⊕ $H(X)$
    7. **O output é $X || Y$.**

  Assim a mensagem pode ser agora cifrada pelo RSA. A propriedade determinística do RSA é evitada usando o encoding OAEP.

  - Para descodificar (*unpadding*):
  
    1. Recuperar a random string *r* = Y ⊕ $H(X)$
    2. Recuperar a messagem *m00...0* = X ⊕ $G(r)$


- **Passos da algoritmia do RSA:**

  1. Gerar primos aleatórios pela utilização da função do **SageMath** que nos oferece um primo até ao limite superior (1º argumento), bem como com o limite inferior (3º argumento). 
    - Assim neste caso temos um primo $2^{b-1} < primo < 2^b-1$
  2. Criação dos primos "*p*" e "*q*", bem como o módulo *n*
  3. Computação rápida de Fórmula de Euler de n, conhecendo *p* e *q*.
    - Assim, $\varphi(n) = (p − 1)(q − 1)$
  4. Criação do ring dos inteiros modulo phi e a escolha aleatória dum inteiro para ser o *e*. 
    - Assim, escolhe-se um inteiro que $1 < e < φ(n)$ e $gcd(e, φ(n)) = 1$.    
    - *e* e $φ(n)$ são co-primos.
  5. Criação do *d*. Como $d \equiv e^{-1} \pmod{\varphi(n)}$, então utilizamos o algoritmo Euclideano extendido para passar para esta forma:
$1=de−k⋅\varphi(n)$ e assim fica na identidade de *Bézout*: $g = gcd(x,y) = sx + ty$.
    - Assim, o trio da variável vai ser $(1, d, -k).

  **Com todos os números criados, temos assim as chaves criadas em *SageMath*.**


Estando todos estes algoritmos definidos e entendidos, desenvolveram-se os metodos necessários para cada classe Python e através de um mini teste pode-se verificar a verdade de toda esta implementação.

## Resolução do Exercício

### Classe Python **OAEP**

In [1]:
import random

class OAEP():
    
    # Função que inicializa toda a instância e os valores globais necessários à sua execução
    # Recebe o valor do módulo de n como parâmetro.
    def __init__(self, n):
        
        self.n = n # Tamanho do RSA modulus (key length)
        self.k0 = 128 # Tamanho-bits da string r (random)
        
    
    def string_to_strbits(self, string):
        
        strbits = ''.join(format(ord(i), 'b') for i in string)
        
        return strbits
    
    def strbits_to_string(self, strbits):

        string = ''

        for i in range(0, len(strbits), 7): 

            tempcharbit = strbits[i:i + 7] 
            decimalChar = int(tempcharbit, 2)
            string = string + chr(decimalChar) 

        return string
        
    def pad(self, mensagem):
        
        # Passar mensagem para String bits
        m_strbits = self.string_to_strbits(mensagem)
        
        # O valor de k1 é n-k0-tamanhoMensagem (para depois fazer padding de zeros, caso seja preciso)
        m_tamanho = len(m_strbits)
        
        self.k1 = self.n-self.k0-m_tamanho

        # Efetuar o padding de zeros à mensagem (String Bits)
        m_strbits_pad = m_strbits + ('0'*self.k1)

        # Criação da string random de k0-bits
        r_number = random.getrandbits(self.k0)
        r_strbits = str(bin(r_number))[2:]
        
        while len(r_strbits) != self.k0:
            r_strbits = r_strbits + '0' # Só para o raro caso de r não ser perto de k0-bits
            
        # Expandir o r para (n-k0) bits (tamanho do X) --- G(r)
        tam_expansao = (self.n - 2*self.k0)
        r_exp_strbits = r_strbits + ('0'*tam_expansao)
        
        # Efetuar o XOR da mensagem pad e o G(r) --- X
        x_strbits = str(bin(int(m_strbits_pad,2) ^^ int(r_exp_strbits,2))[2:]).zfill(len(m_strbits_pad))
        
        # Truncar o X para k0 bits --- H(X)
        h_x_strbits = x_strbits[0:self.k0]
        
        # Efetuar o XOR do r com o H(x) --- Y
        y_strbits = str(bin(int(r_strbits,2) ^^ int(h_x_strbits,2))[2:]).zfill(len(r_strbits))  
                        
        return x_strbits+y_strbits
        
        
    def unpad(self, x, y):
        
        # Truncar o X para k0 bits --- H(X)
        h_x_strbits = x[0:self.k0]
        
        # Efetuar o XOR do Y com o H(x) --- r
        r_strbits = str(bin(int(y,2) ^^ int(h_x_strbits,2))[2:]).zfill(len(y))
        
        # Expandir o r para (n-k0) bits (tamanho do X) --- G(r)
        tam_expansao = (self.n - 2*self.k0)
        g_r_strbits = r_strbits + ('0'*tam_expansao)
        
        # Efetuar o XOR do X com o G(r) --- mensagemPad
        mensagemPad = str(bin(int(x,2) ^^ int(g_r_strbits,2))[2:]).zfill(len(x))
        
        tamMensagem = self.n - self.k0 - self.k1
        
        mensagem_strbits = mensagemPad[0:tamMensagem]
        
        mensagem = self.strbits_to_string(mensagem_strbits)
        
        return mensagem

### Teste Classe Python **OAEP**

In [2]:
myOAEP = OAEP(1024)
rsa = myOAEP.pad("TesteString")

x = rsa[:896]
y = rsa[896:]

mensagem = myOAEP.unpad(x, y)

print "X: {} com o tamanho-bit de {}".format(x, len(x))
print "Y: {} com o tamanho-bit de {}".format(y, len(y))
print "RSA: {} com o tamanho-bit de {}".format(rsa, len(rsa))

print "Mensagem: {}".format(mensagem)

X: 00100100010000100000101000110110011001001111011001110000010001001010110111010000001110101011100011101100001011101110101010110101000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000 com o tamanho-bit de 896
Y: 101010011001011110011111010011001011010011111010011100101101001110111011

### Classe Python **RSA**

In [3]:
class RSA():

    # Função que inicializa toda a instância e os valores globais necessários à sua execução
    def __init__(self, n):
        
        self.keylength = n
    
    def randomprime(self, bits):
        return random_prime(2**bits-1,True,2**(bits-1))
    
    def generatekeys(self):
        
        t = self.keylength/2
        
        self.modulus = 0
        
        while len(str(bin(self.modulus)[2:])) != self.keylength:
            
            # Valor dos primos "p" e "q"
            p = self.randomprime(t)
            q = self.randomprime(t)

            # Valor do Módulo N
            self.modulus = p * q
            
        self.r = IntegerModRing(self.modulus)
            
        
        # Valor do phi(n)
        phi = (p-1)*(q-1)
        
        e = ZZ.random_element(phi)

        while gcd(e, phi) != 1: ## e tem de ser coprimo de phi
            e = ZZ.random_element(phi)

        # G = IntegerModRing(phi)
        # e = G(randomprime(512))

        # O S é igual ao D dado que o inverse multiplicative module é igual a seguir o teorema de bezout.
        # Apenas colocado aqui para nível pedagógico
        # s = 1/e
        bezout = xgcd(e,phi)
        d = Integer(mod(bezout[1],phi));
        
        print "Tamanho do N (key-length): " + str(len(str(bin(self.modulus)[2:])))
        
        # RSA public key
        print "Chave pública: (e: {})".format(e)

        # RSA private key 
        print "Chave privada: (d: {})".format(d)
        
        return (e, d)

    def cifrar(self, m, e):

        a = self.r(m)
        
        # cm = (m ^ e) % self.modulus
        
        cm = a**e
        
        return cm
        
    def decifrar(self, cm, d):

        b = self.r(cm)
        
        # dm = (cm ^ d) % self.modulus
        
        dm = b**d
        
        return dm

### Teste Classe Python **RSA**

In [4]:
tamanhoN = 256


'''
rsamsg = int(rsamsg_strbits, 2)

myRSA = RSA(tamanhoN)

(pubkey, privkey) = myRSA.generatekeys()

ciphertext = myRSA.cifrar(rsamsg, pubkey)

print ciphertext

rsamensagem = myRSA.decifrar(ciphertext, privkey)

print rsamensagem

rsamensagem_strbits = str(bin(int(rsamensagem,2))[2:]).zfill(len(rsamensagem))

x = rsamensagem[:128]
y = rsamensagem[128:]

mensagemfinal = myOAEP.unpad(x,y)

print mensagemfinal

'''

myRSA = RSA(tamanhoN)

(pubkey, privkey) = myRSA.generatekeys()

ciphertext = myRSA.cifrar(110, pubkey)

mensagem = myRSA.decifrar(ciphertext, privkey)

print "RSA funcionou? ",mensagem == 110

Tamanho do N (key-length): 256
Chave pública: (e: 45442704347296726502485820465662804526282074652382115438007791410159709848499)
Chave privada: (d: 50635373912691781259367731963541570719007932522770514866395590312930160020887)
RSA funcionou?  True


## Referências

https://en.wikipedia.org/wiki/Optimal_asymmetric_encryption_padding

https://en.wikipedia.org/wiki/Ciphertext_indistinguishability

https://en.wikipedia.org/wiki/RSA_(cryptosystem)